<a href="https://colab.research.google.com/github/RVC-Boss/GPT-SoVITS/blob/main/colab_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

环境配置 environment

In [ ]:
!pip install -q condacolab
# Setting up condacolab and installing packages
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py39_23.11.0-2-Linux-x86_64.sh")
%cd -q /content
!git clone https://github.com/AlexanderLJX/GPT-SoVITS
!conda install -y -q -c pytorch -c nvidia cudatoolkit
%cd -q /content/GPT-SoVITS
!conda install -y -q -c conda-forge gcc gxx ffmpeg cmake -c pytorch -c nvidia
!/usr/local/bin/pip install -r requirements.txt

In [ ]:
# @title Download pretrained models 下载预训练模型
!mkdir -p /content/GPT-SoVITS/GPT_SoVITS/pretrained_models
!mkdir -p /content/GPT-SoVITS/tools/damo_asr/models
!mkdir -p /content/GPT-SoVITS/tools/uvr5
%cd /content/GPT-SoVITS/GPT_SoVITS/pretrained_models
!git clone https://huggingface.co/lj1995/GPT-SoVITS
%cd /content/GPT-SoVITS/tools/damo_asr/models
!git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
!git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
!git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git
# @title UVR5 pretrains 安装uvr5模型
%cd /content/GPT-SoVITS/tools/uvr5
!git clone https://huggingface.co/Delik/uvr5_weights
!git config core.sparseCheckout true
!mv /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/

In [ ]:
# @title launch WebUI 启动WebUI
!/usr/local/bin/pip install ipykernel
!sed -i '9s/False/True/' /content/GPT-SoVITS/config.py
%cd /content/GPT-SoVITS/

In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nServer finished loading, trying to launch localtunnel\n")

  print("The public IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')

# Adjust the port number if your api.py uses a different port
threading.Thread(target=iframe_thread, daemon=True, args=(9880,)).start()

# Ensure api.py is executable and uses the correct port
!python api.py